In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import ensemble
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn import preprocessing

In [2]:
dataTrain = pd.read_csv('Train_UWu5bXk.txt')
dataTest = pd.read_csv('Test_u94Q5KV.txt')

In [3]:
dataTest.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [4]:
np.unique(dataTrain[['Item_Fat_Content']].values)

array(['LF', 'Low Fat', 'Regular', 'low fat', 'reg'], dtype=object)

In [5]:
np.unique(dataTest[['Item_Fat_Content']].values)

array(['LF', 'Low Fat', 'Regular', 'low fat', 'reg'], dtype=object)

In [6]:
np.unique(dataTest[['Item_Type']].values)

array(['Baking Goods', 'Breads', 'Breakfast', 'Canned', 'Dairy',
       'Frozen Foods', 'Fruits and Vegetables', 'Hard Drinks',
       'Health and Hygiene', 'Household', 'Meat', 'Others', 'Seafood',
       'Snack Foods', 'Soft Drinks', 'Starchy Foods'], dtype=object)

In [7]:
np.unique(dataTrain[['Item_Type']].values)

array(['Baking Goods', 'Breads', 'Breakfast', 'Canned', 'Dairy',
       'Frozen Foods', 'Fruits and Vegetables', 'Hard Drinks',
       'Health and Hygiene', 'Household', 'Meat', 'Others', 'Seafood',
       'Snack Foods', 'Soft Drinks', 'Starchy Foods'], dtype=object)

In [8]:
np.unique(dataTest[['Outlet_Identifier']].values)

array(['OUT010', 'OUT013', 'OUT017', 'OUT018', 'OUT019', 'OUT027',
       'OUT035', 'OUT045', 'OUT046', 'OUT049'], dtype=object)

In [9]:
np.unique(dataTrain[['Outlet_Identifier']].values)

array(['OUT010', 'OUT013', 'OUT017', 'OUT018', 'OUT019', 'OUT027',
       'OUT035', 'OUT045', 'OUT046', 'OUT049'], dtype=object)

In [10]:
np.unique(dataTrain[['Outlet_Location_Type','Outlet_Type']].values)

array(['Grocery Store', 'Supermarket Type1', 'Supermarket Type2',
       'Supermarket Type3', 'Tier 1', 'Tier 2', 'Tier 3'], dtype=object)

In [11]:
np.unique(dataTest[['Outlet_Location_Type','Outlet_Type']].values)

array(['Grocery Store', 'Supermarket Type1', 'Supermarket Type2',
       'Supermarket Type3', 'Tier 1', 'Tier 2', 'Tier 3'], dtype=object)

In [12]:
dataTrain.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [13]:
dataTest.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [14]:
dataTest['Outlet_Size'].fillna('U',inplace = True)
dataTrain['Outlet_Size'].fillna('U',inplace = True)
dataTrain['Item_Weight'].replace(np.nan,dataTrain['Item_Weight'].mean(),inplace=True)
dataTest['Item_Weight'].replace(np.nan,dataTest['Item_Weight'].mean(),inplace=True)

dataTrain['Outlet_Years'] = 2013 - dataTrain['Outlet_Establishment_Year']
dataTest['Outlet_Years'] = 2013 - dataTest['Outlet_Establishment_Year']

In [15]:
dataTest.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Outlet_Years                 0
dtype: int64

In [16]:
dataTrain.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
Outlet_Years                 0
dtype: int64

In [17]:
def educationEncodingAndFilter(x):
    if (x=='LF' or x == 'Low Fat' or x=='low fat'):
        return 0
    else:
        return 1

In [18]:
dataTrain['Item_Fat_Content']=dataTrain['Item_Fat_Content'].apply(educationEncodingAndFilter)
dataTrain.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Years
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,14
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,4
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,14
3,FDX07,19.20,1,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,U,Tier 3,Grocery Store,732.3800,15
4,NCD19,8.93,0,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,26


In [19]:
dataTest['Item_Fat_Content']=dataTest['Item_Fat_Content'].apply(educationEncodingAndFilter)
dataTest.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Outlet_Years
0,FDW58,20.750000,0,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,14
1,FDW14,8.300000,1,0.038428,Dairy,87.3198,OUT017,2007,U,Tier 2,Supermarket Type1,6
2,NCN55,14.600000,0,0.099575,Others,241.7538,OUT010,1998,U,Tier 3,Grocery Store,15
3,FDQ58,7.315000,0,0.015388,Snack Foods,155.0340,OUT017,2007,U,Tier 2,Supermarket Type1,6
4,FDY38,12.695633,1,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,28


In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#New variable for outlet
dataTrain['Outlet'] = le.fit_transform(dataTrain['Outlet_Identifier'])
dataTest['Outlet'] = le.fit_transform(dataTest['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Outlet','Item_Type']
le = LabelEncoder()
for i in var_mod:
    dataTrain[i] = le.fit_transform(dataTrain[i])
le = LabelEncoder()
for i in var_mod:
    dataTest[i] = le.fit_transform(dataTest[i])

In [21]:
dataTrain.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Outlet_Years,Outlet
0,FDA15,9.30,0,0.016047,4,249.8092,OUT049,1999,1,0,1,3735.1380,14,9
1,DRC01,5.92,1,0.019278,14,48.2692,OUT018,2009,1,2,2,443.4228,4,3
2,FDN15,17.50,0,0.016760,10,141.6180,OUT049,1999,1,0,1,2097.2700,14,9
3,FDX07,19.20,1,0.000000,6,182.0950,OUT010,1998,3,2,0,732.3800,15,0
4,NCD19,8.93,0,0.000000,9,53.8614,OUT013,1987,0,2,1,994.7052,26,1


In [22]:
dataTrain = pd.get_dummies(dataTrain, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                            'Outlet','Item_Type'])
dataTest = pd.get_dummies(dataTest, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                            'Outlet','Item_Type'])

In [23]:
dataTrain.head()
dataTrain.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Size_3                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Outlet_0                       uint8
Outlet_1                       uint8
Outlet_2                       uint8
Outlet_3                       uint8
Outlet_4                       uint8
Outlet_5                       uint8
O

In [24]:
dataTrain.drop(['Outlet_Establishment_Year'],axis=1,inplace=True)

In [25]:
dataTrain.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,...,Item_Type_6,Item_Type_7,Item_Type_8,Item_Type_9,Item_Type_10,Item_Type_11,Item_Type_12,Item_Type_13,Item_Type_14,Item_Type_15
0,FDA15,9.30,0.016047,249.8092,OUT049,3735.1380,14,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,DRC01,5.92,0.019278,48.2692,OUT018,443.4228,4,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,FDN15,17.50,0.016760,141.6180,OUT049,2097.2700,14,1,0,1,...,0,0,0,0,1,0,0,0,0,0
3,FDX07,19.20,0.000000,182.0950,OUT010,732.3800,15,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.000000,53.8614,OUT013,994.7052,26,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [26]:
dataTest.drop(['Outlet_Establishment_Year'],axis=1,inplace=True)

In [27]:
dataTest.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,Outlet_Location_Type_1,...,Item_Type_6,Item_Type_7,Item_Type_8,Item_Type_9,Item_Type_10,Item_Type_11,Item_Type_12,Item_Type_13,Item_Type_14,Item_Type_15
0,FDW58,20.750000,0.007565,107.8622,OUT049,14,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,FDW14,8.300000,0.038428,87.3198,OUT017,6,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,NCN55,14.600000,0.099575,241.7538,OUT010,15,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,FDQ58,7.315000,0.015388,155.0340,OUT017,6,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,FDY38,12.695633,0.118599,234.2300,OUT027,28,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
yTrain=dataTrain['Item_Outlet_Sales']
dataTrain.drop(labels=['Item_Outlet_Sales','Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [29]:
yTest=dataTest['Item_Identifier']
dataTest.drop(labels=['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [31]:
preds = pd.DataFrame()
m2 = ensemble.ExtraTreesRegressor(n_estimators=550)  
m2.fit(dataTrain,yTrain)
preds["Model2"] = m2.predict(dataTest)

In [32]:
from xgboost import XGBRegressor
m3 = XGBRegressor(max_depth=20, n_estimators=1000)  
m3.fit(dataTrain,yTrain)
preds["Model3"] = m3.predict(dataTest)

C:\Users\shivam\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\shivam\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [33]:
m4 = ensemble.RandomForestRegressor(n_estimators=170)
m4.fit(dataTrain,yTrain)
preds["Model4"] = m4.predict(dataTest)

In [34]:
preds.head()

,Model2,Model3,Model4
0,1796.842882,1380.596069,1762.110196
1,1479.345862,1091.279785,1255.036916
2,496.724327,724.736572,655.918745
3,1525.365958,2778.453125,2071.875605
4,6010.241969,6425.494629,6069.941941


In [35]:
preds['avg'] = preds.mean(axis=1)

In [36]:
test = pd.read_csv('Test_u94Q5KV.txt')

In [37]:
yTest1 = test['Outlet_Identifier']

In [38]:
sub = pd.DataFrame({"Item_Identifier": yTest,"Outlet_Identifier":yTest1,"Item_Outlet_Sales": preds['Model2']})
sub.to_csv("sub.csv", index=False)

In [39]:
sub = pd.DataFrame({"Item_Identifier": yTest,"Outlet_Identifier":yTest1,"Item_Outlet_Sales": preds['Model3']})
sub.to_csv("sub1.csv", index=False)

In [40]:
sub = pd.DataFrame({"Item_Identifier": yTest,"Outlet_Identifier":yTest1,"Item_Outlet_Sales": preds['Model4']})
sub.to_csv("sub2.csv", index=False)

In [41]:
sub = pd.DataFrame({"Item_Identifier": yTest,"Outlet_Identifier":yTest1,"Item_Outlet_Sales": preds['avg']})
sub.to_csv("sub3.csv", index=False)